**Download the Dataset**

In [ ]:
!pip install roboflow

from roboflow import Roboflow8
import os
rf = Roboflow(api_key=os.environ["ROBOFLOW_APIKEY"])
project = rf.workspace("junaid-bashir-bfjgs").project("rebarcounterstemeye")
version = project.version(1)
dataset = version.download("voc")


**Install the required packages**

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

**Import the required packages.**

In [ ]:
import numpy as np
import os
from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
from tflite_support import metadata
import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

**Load the dataset**

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'train/images',
    'train/xml',
    ['label']
)
val_data = object_detector.DataLoader.from_pascal_voc(
    'validate/images',
    'validate/xml',
    ['label']
)

**Select a model architecture**

In [ ]:
spec = object_detector.EfficientDetSpec(
  model_name='efficientdet-lite4',
  uri='https://tfhub.dev/tensorflow/efficientdet/lite4/feature-vector/2',
  hparams={'max_instances_per_image': 800},
  tflite_max_detections=400
  )

**Train the model with the training data.**

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=16, train_whole_model=True, epochs=20, validation_data=val_data)

**Export as a tflite model.**

In [ ]:
model.export(export_dir='.', tflite_filename='rebarcounter.tflite')